# Fourier Descriptors

In [37]:
# Import all the necessary libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from typing import Callable
import cv2 as cv
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects
import pandas as pd
import pickle

#### Load pickle

In [39]:
def load_pickle(file_name, load_path):
    """Load a variable from a binary format path

    Args:
        file_path: the file path where the file is stored

    Returns:
        return the content of the file, generally a dataFrame here.
    """
    file_path = os.path.join(load_path, file_name)
    with open(file_path, "rb") as file:
        return pickle.load(file)
    
images = load_pickle("coins.pkl", "../data/")

In [40]:
def convert_gray(image):
    """Convert an image to gray

    Args:
        image: the image to convert

    Returns:
        return the converted image
    """
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

def find_contour(images: np.ndarray):
    """
    Find the contours for the set of images
    
    Args
    ----
    images: np.ndarray (N, 28, 28)
        Source images to process

    Return
    ------
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    """

    # Get number of images to process
    N, _, _ = np.shape(images)
    # Fill in dummy values (fake points)
    contours = [np.array([[0, 0], [1, 1]]) for i in range(N)]

    # ------------------
    # Your code here ... 
    for idx, i in enumerate(images):
        cnts = sk.measure.find_contours(np.transpose(i), fully_connected='high')[0]
        contours[idx] = cnts
    # ------------------
    
    return contours
    
def linear_interpolation(contours: np.ndarray, n_samples: int = 11):
    """
    Perform interpolation/resampling of the contour across n_samples.
    
    Args
    ----
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    n_samples: int
        Number of samples to consider along the contour.

    Return
    ------
    contours_inter: np.ndarray (N, n_samples, 2)
        Interpolated contour with n_samples
    """

    N = len(contours)
    contours_inter = np.zeros((N, n_samples, 2))
    
    # ------------------
    # Your code here ... 
    for idx, cnt in enumerate(contours):
        differences = np.diff(cnt, axis=0)
        distances = np.sqrt(np.sum(differences**2, axis=1))
        cumulative_distances = np.concatenate([[0], np.cumsum(distances)])

        # Interpolate the x and y coordinates
        t = np.linspace(0, cumulative_distances[-1], n_samples + 1)
        t = t[:-1]
        x = np.interp(t, cumulative_distances, cnt[:, 0])
        y = np.interp(t, cumulative_distances, cnt[:, 1])
        contours_inter[idx] = np.stack([x, y], axis=1)
    # ------------------
        
    return contours_inter

def compute_descriptor_padding(contours: np.ndarray, n_samples: int = 11):
    """
    Compute Fourier descriptors of input images
    
    Args
    ----
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    n_samples: int
        Number of samples to consider. If the contour length is higher, discard the remaining part. If it is shorter, add padding.
        Make sure that the first element of the descriptor represents the continuous component.

    Return
    ------
    descriptors: np.ndarray complex (N, n_samples)
        Computed complex Fourier descriptors for the given input images
    """

    N = len(contours)
    # Look for the number of contours
    descriptors = np.zeros((N, n_samples), dtype=np.complex_)

    # ------------------
    # Your code here ... 
    descriptors = np.array([np.fft.fft(contours[i][:, 0] + 1j * contours[i][:, 1], n_samples) for i in range(N)])
    # ------------------

    return descriptors

def plot_features(features_a: np.ndarray, features_b: np.ndarray, label_a: str, label_b: str, title: str):
    """
    Plot feature components a and b.
    
    Args
    ----
    features_a: np.ndarray (N, D)
        Feature a with N samples and D complex features. 
    features_b: np.ndarray (N, D)
        Feature b with N samples and D complex features.
    label_a: str
        Name of the feature a.
    label_b: str
        Name of the feature b.
    """

    # Number of paris to display
    n_features = features_a.shape[1]
    # Define pairs for 2D plots
    pairs = np.array(range(2*np.ceil(n_features / 2).astype(int)))
    # Check if odd lenght, shift second feature to have pairs
    if n_features % 2 == 1:
        pairs[2:] = pairs[1:-1]
    # Convert to 2d array
    pairs = pairs.reshape(-1, 2)

    # Plot each pairs and labels
    n_plots = len(pairs)
    _, axes = plt.subplots(3, n_plots, figsize=(15, 8))
    
    for i, (pa, pb) in enumerate(pairs):
        # Real
        axes[0, i].scatter(np.real(features_a[:, pa]), np.real(features_a[:, pb]), label=label_a, s=10, alpha=0.1)
        axes[0, i].scatter(np.real(features_b[:, pa]), np.real(features_b[:, pb]), label=label_b, s=10, alpha=0.1)
        axes[0, i].set_xlabel("Component {}".format(pa))
        axes[0, i].set_ylabel("Component {}".format(pb))
        axes[0, i].set_title("Real {} vs {}".format(pa, pb))
        axes[0, i].legend()
        # Imag
        axes[1, i].scatter(np.imag(features_a[:, pa]), np.imag(features_a[:, pb]), label=label_a, s=10, alpha=0.1)
        axes[1, i].scatter(np.imag(features_b[:, pa]), np.imag(features_b[:, pb]), label=label_b, s=10, alpha=0.1)
        axes[1, i].set_xlabel("Component {}".format(pa))
        axes[1, i].set_ylabel("Component {}".format(pb))
        axes[1, i].set_title("Imag. {} vs {}".format(pa, pb))
        axes[1, i].legend()
        # Abs
        axes[2, i].scatter(np.abs(features_a[:, pa]), np.abs(features_a[:, pb]), label=label_a, s=10, alpha=0.1)
        axes[2, i].scatter(np.abs(features_b[:, pa]), np.abs(features_b[:, pb]), label=label_b, s=10, alpha=0.1)
        axes[2, i].set_xlabel("Component {}".format(pa))
        axes[2, i].set_ylabel("Component {}".format(pb))
        axes[2, i].set_title("Abs. {} vs {}".format(pa, pb))
        axes[2, i].legend()

    plt.suptitle(title)
    plt.tight_layout()

def resize_images(images, size):
    """Resize the images
    Args:
        images (list): list of images
        size (tuple): size of the images
    Returns:

        np.array: resized images
    """
    resized_images = [cv.resize(image, (size,size)) for image in images]
    return np.array(resized_images)

def pad_images(images):
    """Pad the images to the biggest image
    Args:
        images (list): list of images
    
    Returns:
        images_padded : list of padded images
    """

    # Find the dimensions of the biggest image
    biggest_dim = max(max(image.shape[:2]) for image in images)
    images_padded = []
    for img in images:
        # Calculate padding dimensions
        height, width = img.shape[:2]
        pad_height = biggest_dim - height
        pad_width = biggest_dim - width

        # Distribute padding evenly on both sides of the image
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left

        # Pad image with black
        img_padded = cv.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv.BORDER_CONSTANT, value=[0, 0, 0])
        images_padded.append(img_padded)
    
    return images_padded


In [55]:
N_SAMPLES = 11

# images groups and convert to dict
image_group_1 = []
image_group_2 = []

# pad images
image_group_1_padded = pad_images(image_group_1)
image_group_2_padded = pad_images(image_group_2)

# resize images
image_group_1_resized = resize_images(image_group_1_padded, 300)
image_group_2_resized = resize_images(image_group_2_padded, 300)

print(image_group_1_resized[0].shape)

# convert images to gray
image_group_1_gray = [convert_gray(image) for image in image_group_1_resized]
image_group_2_gray = [convert_gray(image) for image in image_group_2_resized]

# find contours
contours_group_1 = find_contour(image_group_1_gray)
contours_group_2 = find_contour(image_group_2_gray)

# interpolate contours
contours_inter_group_1 = linear_interpolation(contours_group_1, N_SAMPLES)
contours_inter_group_2 = linear_interpolation(contours_group_2, N_SAMPLES)

# compute descriptors
descriptors_group_1 = compute_descriptor_padding(contours_inter_group_1, N_SAMPLES)
descriptors_group_2 = compute_descriptor_padding(contours_inter_group_2, N_SAMPLES)

# plot features
plot_features(descriptors_group_1, descriptors_group_2, "Group 1", "Group 2", "Fourier descriptors")


AttributeError: 'str' object has no attribute 'shape'